# Test de connexion et permissions SQL Server

Ce notebook teste les droits d'accès à la base de données SQL Server en :
1. Se connectant à la base de données
2. Créant une table de test
3. Insérant des données
4. Lisant les données
5. Mettant à jour les données
6. Supprimant les données
7. Supprimant la table de test

In [18]:
import pyodbc
import os
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables
load_dotenv()

# Get connection parameters from .env
SERVER = os.getenv('SERVER_NAME')
DATABASE = os.getenv('DATABASE_NAME')
USER = os.getenv('SQL_LOGIN_USER')
PASSWORD = os.getenv('SQL_LOGIN_PASSWORD')

print(f"Configuration:")
print(f"  Server: {SERVER}")
print(f"  Database: {DATABASE}")
print(f"  User: {USER}")
print(f"  Password: {'*' * len(PASSWORD) if PASSWORD else 'NOT SET'}")

Configuration:
  Server: ZEUS11
  Database: france_172074
  User: user_breau
  Password: ****************


## 1. Test de connexion

In [12]:
# Connection string with SQL Server authentication
DRIVER = '{ODBC Driver 17 for SQL Server}'
connection_string = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};UID={USER};PWD={PASSWORD};Encrypt=no;'

try:
    conn = pyodbc.connect(connection_string)
    print("✅ Connexion réussie !")
    
    # Get SQL Server version
    cursor = conn.cursor()
    cursor.execute("SELECT @@VERSION")
    version_row = cursor.fetchone()
    if version_row:
        version = version_row[0]
        print(f"\n📋 Version SQL Server:\n{version.split(chr(10))[0]}")
    
    # Get current database
    cursor.execute("SELECT DB_NAME()")
    db_row = cursor.fetchone()
    if db_row:
        print(f"\n📁 Base de données courante: {db_row[0]}")
    
    cursor.close()
    
except Exception as e:
    print(f"❌ Erreur de connexion: {e}")
    conn = None

✅ Connexion réussie !

📋 Version SQL Server:
Microsoft SQL Server 2017 (RTM-CU31-GDR) (KB5065225) - 14.0.3505.1 (X64) 

📁 Base de données courante: france_172074


## 2. Test de création de table (CREATE)

In [13]:
if conn:
    cursor = conn.cursor()
    
    # Drop table if it exists (from previous test)
    try:
        cursor.execute("DROP TABLE IF EXISTS test_permissions")
        conn.commit()
        print("🧹 Table de test existante supprimée (si elle existait)")
    except Exception as e:
        print(f"⚠️  Info: {e}")
        conn.rollback()
    
    # Create test table
    try:
        cursor.execute("""
            CREATE TABLE test_permissions (
                id INT PRIMARY KEY IDENTITY(1,1),
                name NVARCHAR(100) NOT NULL,
                test_date DATETIME DEFAULT GETDATE(),
                description NVARCHAR(255)
            )
        """)
        conn.commit()
        print("✅ Table 'test_permissions' créée avec succès")
        print("   ├─ Colonne: id (INT, PRIMARY KEY, IDENTITY)")
        print("   ├─ Colonne: name (NVARCHAR(100))")
        print("   ├─ Colonne: test_date (DATETIME)")
        print("   └─ Colonne: description (NVARCHAR(255))")
    except Exception as e:
        print(f"❌ Erreur lors de la création de table: {e}")
        conn.rollback()
    
    cursor.close()
else:
    print("❌ Pas de connexion active")

🧹 Table de test existante supprimée (si elle existait)
✅ Table 'test_permissions' créée avec succès
   ├─ Colonne: id (INT, PRIMARY KEY, IDENTITY)
   ├─ Colonne: name (NVARCHAR(100))
   ├─ Colonne: test_date (DATETIME)
   └─ Colonne: description (NVARCHAR(255))


## 3. Test d'insertion de données (INSERT)

In [14]:
if conn:
    cursor = conn.cursor()
    
    try:
        # Insert test data
        test_data = [
            ('Test Record 1', 'Premier enregistrement de test'),
            ('Test Record 2', 'Deuxième enregistrement de test'),
            ('Test Record 3', 'Troisième enregistrement de test')
        ]
        
        cursor.executemany(
            "INSERT INTO test_permissions (name, description) VALUES (?, ?)",
            test_data
        )
        conn.commit()
        print(f"✅ {len(test_data)} lignes insérées avec succès")
        for i, (name, desc) in enumerate(test_data, 1):
            print(f"   {i}. {name} - {desc}")
    except Exception as e:
        print(f"❌ Erreur lors de l'insertion: {e}")
        conn.rollback()
    
    cursor.close()
else:
    print("❌ Pas de connexion active")

✅ 3 lignes insérées avec succès
   1. Test Record 1 - Premier enregistrement de test
   2. Test Record 2 - Deuxième enregistrement de test
   3. Test Record 3 - Troisième enregistrement de test


## 4. Test de lecture de données (SELECT)

In [5]:
if conn:
    cursor = conn.cursor()
    
    try:
        cursor.execute("SELECT id, name, test_date, description FROM test_permissions ORDER BY id")
        rows = cursor.fetchall()
        
        print(f"✅ {len(rows)} lignes lues avec succès:\n")
        print(f"{'ID':<5} {'Name':<20} {'Date':<25} {'Description'}")
        print("-" * 80)
        for row in rows:
            date_str = row[2].strftime('%Y-%m-%d %H:%M:%S') if row[2] else 'N/A'
            print(f"{row[0]:<5} {row[1]:<20} {date_str:<25} {row[3] or ''}")
    except Exception as e:
        print(f"❌ Erreur lors de la lecture: {e}")
    
    cursor.close()
else:
    print("❌ Pas de connexion active")

✅ 3 lignes lues avec succès:

ID    Name                 Date                      Description
--------------------------------------------------------------------------------
1     Test Record 1        2025-11-17 10:29:54       Premier enregistrement de test
2     Test Record 2        2025-11-17 10:29:54       Deuxième enregistrement de test
3     Test Record 3        2025-11-17 10:29:54       Troisième enregistrement de test


## 5. Test de mise à jour de données (UPDATE)

In [6]:
if conn:
    cursor = conn.cursor()
    
    try:
        # Update first record
        cursor.execute("""
            UPDATE test_permissions 
            SET name = 'Test Record 1 - UPDATED',
                description = 'Cette ligne a été mise à jour avec succès'
            WHERE id = 1
        """)
        conn.commit()
        print(f"✅ Ligne mise à jour avec succès ({cursor.rowcount} ligne(s) affectée(s))")
        
        # Verify update
        cursor.execute("SELECT id, name, description FROM test_permissions WHERE id = 1")
        row = cursor.fetchone()
        if row:
            print(f"\n📝 Nouvelle valeur:")
            print(f"   ID: {row[0]}")
            print(f"   Name: {row[1]}")
            print(f"   Description: {row[2]}")
    except Exception as e:
        print(f"❌ Erreur lors de la mise à jour: {e}")
        conn.rollback()
    
    cursor.close()
else:
    print("❌ Pas de connexion active")

✅ Ligne mise à jour avec succès (1 ligne(s) affectée(s))

📝 Nouvelle valeur:
   ID: 1
   Name: Test Record 1 - UPDATED
   Description: Cette ligne a été mise à jour avec succès


## 6. Test de suppression de données (DELETE)

In [15]:
if conn:
    cursor = conn.cursor()
    
    try:
        # Delete one record
        cursor.execute("DELETE FROM test_permissions WHERE id = 2")
        conn.commit()
        print(f"✅ Ligne supprimée avec succès ({cursor.rowcount} ligne(s) affectée(s))")
        
        # Verify remaining rows
        cursor.execute("SELECT COUNT(*) FROM test_permissions")
        count_row = cursor.fetchone()
        count = count_row[0] if count_row else 0
        print(f"\n📊 Lignes restantes: {count}")
        
        # Show remaining records
        cursor.execute("SELECT id, name FROM test_permissions ORDER BY id")
        rows = cursor.fetchall()
        if rows:
            print("\n📋 Enregistrements restants:")
            for row in rows:
                print(f"   - ID {row[0]}: {row[1]}")
    except Exception as e:
        print(f"❌ Erreur lors de la suppression: {e}")
        conn.rollback()
    
    cursor.close()
else:
    print("❌ Pas de connexion active")

✅ Ligne supprimée avec succès (1 ligne(s) affectée(s))

📊 Lignes restantes: 2

📋 Enregistrements restants:
   - ID 1: Test Record 1
   - ID 3: Test Record 3


## 7. Nettoyage - Suppression de la table de test (DROP)

In [16]:
if conn:
    cursor = conn.cursor()
    
    try:
        cursor.execute("DROP TABLE test_permissions")
        conn.commit()
        print("✅ Table 'test_permissions' supprimée avec succès")
        print("\n🧹 Nettoyage terminé - aucune trace laissée dans la base de données")
    except Exception as e:
        print(f"❌ Erreur lors de la suppression de table: {e}")
        conn.rollback()
    
    cursor.close()
else:
    print("❌ Pas de connexion active")

✅ Table 'test_permissions' supprimée avec succès

🧹 Nettoyage terminé - aucune trace laissée dans la base de données


## 8. Fermeture de la connexion

In [19]:
if conn:
    conn.close()
    print("✅ Connexion fermée proprement")
    print("\n" + "="*70)
    print("TESTS TERMINÉS")
    print("="*70)
else:
    print("⚠️  Pas de connexion à fermer")

ProgrammingError: Attempt to use a closed connection.

---

## 📊 Résumé des tests de permissions

Si tous les tests ci-dessus affichent ✅, alors vous avez **tous les droits nécessaires** :

| Permission | Status | Description |
|------------|--------|-------------|
| **CONNECT** | ✅ | Vous pouvez vous connecter à la base de données |
| **CREATE** | ✅ | Vous pouvez créer des tables |
| **INSERT** | ✅ | Vous pouvez insérer des données |
| **SELECT** | ✅ | Vous pouvez lire des données |
| **UPDATE** | ✅ | Vous pouvez mettre à jour des données |
| **DELETE** | ✅ | Vous pouvez supprimer des données |
| **DROP** | ✅ | Vous pouvez supprimer des tables |

---

### 🔧 En cas de problème

Si certains tests échouent :

1. **Erreur de connexion** : Vérifiez le nom du serveur, les credentials dans le `.env`
2. **CREATE échoue** : Vous n'avez pas les droits de création de tables
3. **INSERT/UPDATE/DELETE échoue** : Vous n'avez pas les droits DML sur cette table
4. **DROP échoue** : Vous n'avez pas les droits de suppression de tables

Contactez votre administrateur de base de données pour obtenir les permissions manquantes.